In [1]:
import random

CUSTOMERS = [(0, 0), (1, 5), (2, 2), (3, 8), (5, 5), (6, 1), (8, 3)]

POPULATION_SIZE = 4  # Smaller population for demonstration
GENERATIONS = 2
VEHICLES = 3

def create_individual():
    """
    Creates a random individual for a multi-vehicle TSP/VRP.
    Representation: List of lists, one list per vehicle containing assigned customers.
    Allows uneven distribution of customers.
    """
    customers = list(range(1, len(CUSTOMERS)))  # exclude depot (index 0)
    random.shuffle(customers)

    individual = [[] for _ in range(VEHICLES)]

    for cust in customers:
        chosen_vehicle = random.randint(0, VEHICLES - 1)  # assign randomly
        individual[chosen_vehicle].append(cust)

    return individual

In [2]:
# Testing creation of individuals
INDIVIDUALS = []
for i in range(POPULATION_SIZE):
  INDIVIDUALS.append(create_individual())

INDIVIDUALS

[[[6, 2, 4, 1], [5], [3]],
 [[1, 6], [2], [3, 4, 5]],
 [[3], [], [1, 2, 6, 5, 4]],
 [[4], [5, 1], [3, 2, 6]]]

In [3]:
def fitness(individual):
    individual = decode_individual(individual) #adding depot for start and end
    total_distance = 0

    for route in individual:  # go through each vehicle's route
        if len(route) > 1:  # if vehicle visits more than one customer
            for i in range(1, len(route)):
                x1, y1 = CUSTOMERS[route[i-1]]
                x2, y2 = CUSTOMERS[route[i]]
                total_distance += ((x2 - x1)**2 + (y2 - y1)**2)**0.5

    return 1 / total_distance if total_distance > 0 else 0


def decode_individual(individual):
    return [[0] + route + [0] for route in individual]

In [4]:
FITNESSES = []
for individual in INDIVIDUALS:
  FITNESSES.append(fitness(individual))

FITNESSES

[0.017475808535642384,
 0.020435099823675223,
 0.021999940805808166,
 0.016398664802950425]

In [5]:
def tournament_selection(population, fitnesses, k=3):
    # pick k random indices
    participants = random.sample(range(len(population)), k)
    print("participants: ", participants)
    
    # find the best among them
    best_idx = participants[0]
    for idx in participants[1:]:
        if fitnesses[idx] > fitnesses[best_idx]:
            best_idx = idx
    
    return population[best_idx]

In [6]:
selected = tournament_selection(INDIVIDUALS, FITNESSES)
print("Selected individual:", selected)

participants:  [3, 0, 1]
Selected individual: [[1, 6], [2], [3, 4, 5]]


In [7]:
def swap_mutation(individual):
    vehicle = random.choice([v for v in individual if len(v) > 1])
    i, j = random.sample(range(len(vehicle)), 2)
    vehicle[i], vehicle[j] = vehicle[j], vehicle[i]
    print("swap")
    return individual


def relocation_mutation(individual):
    # pick a non-empty vehicle as source
    from_vehicle = random.choice([v for v in individual if v])
    cust = from_vehicle.pop(random.randrange(len(from_vehicle)))

    # pick a different vehicle as target
    candidates = [v for v in individual if v is not from_vehicle]
    to_vehicle = random.choice(candidates)

    # insert at random position
    insert_pos = random.randint(0, len(to_vehicle))
    to_vehicle.insert(insert_pos, cust)

    print("relocation")
    return individual


def mutate(individual):
    chance = random.random()
    if chance <= 1:  # 10% mutation rate for demonstration
      if chance < 0.5: # 50% of each mutation type
        relocation_mutation(individual)
      else: swap_mutation(individual)
    return individual

In [8]:
for individual in INDIVIDUALS:
  mutate(individual)

INDIVIDUALS

relocation
relocation
relocation
relocation


[[[6, 2, 4], [5], [3, 1]],
 [[1, 6], [2, 3], [4, 5]],
 [[], [], [1, 3, 2, 6, 5, 4]],
 [[6, 4], [5, 1], [3, 2]]]

In [9]:
import random

def order_crossover(parent1, parent2):
    number_of_customers = sum(len(r) for r in parent1)

    # 1. Create lists without vehicle format from parents
    parent1_to_list = [c for route in parent1 for c in route]
    parent2_to_list = [c for route in parent2 for c in route]

    # 2. Pick random crossover points and add to child
    start, end = sorted(random.sample(range(len(parent1_to_list)), 2))
    child_list = [None]*number_of_customers
    child_list[start:end+1] = parent1_to_list[start:end+1]

    # 3. Fill remaining customers from parent2 in order
    pointer = (end + 1) % len(child_list)
    for c in parent2_to_list:
        if c not in child_list:
            child_list[pointer] = c
            pointer = (pointer + 1) % len(child_list)

    # 4. Split child list into vehicle routes based on parent1 format
    child = []
    idx = 0
    for vehicle in parent1:
        route_length = len(vehicle)
        child.append(child_list[idx:idx+route_length])
        idx += route_length

    return child

In [10]:
print(order_crossover(INDIVIDUALS[0], INDIVIDUALS[1]))
print(order_crossover(INDIVIDUALS[1], INDIVIDUALS[0]))

[[5, 2, 4], [1], [6, 3]]
[[1, 6], [2, 4], [5, 3]]
